In [ ]:
import pandas as pd

# Load the CSV files
before_df = pd.read_csv('../results/TTM_CT_Job_Before.csv')
after_df = pd.read_csv('../results/TTM_CT_Job_After.csv')
neighbourhood_df = pd.read_csv('../../../data/visual_data/CTUID-w-Neighborhood.csv')

# Merge with neighbourhood data
before_df = before_df.merge(neighbourhood_df, left_on='from_id', right_on='CTUID')
after_df = after_df.merge(neighbourhood_df, left_on='from_id', right_on='CTUID')

# Calculate the benefit
before_df['Before_After_Benefit'] = 'before'
after_df['Before_After_Benefit'] = 'after'
combined_df = pd.concat([before_df, after_df])

# Calculate the benefit (before - after)
benefit_df = before_df.copy()
benefit_df['Before_After_Benefit'] = 'benefit'
benefit_df['travel_time'] = before_df['travel_time'] - after_df['travel_time']
combined_df = pd.concat([combined_df, benefit_df])

# Add constant columns
combined_df['Metric_Type'] = 'Economic'
combined_df['Metric_Name'] = 'Travel Time to Key Work Locations'
combined_df['Travel_Time_Threshold'] = combined_df['travel_time']

# Rename columns
combined_df.rename(columns={'AREA_NAME': 'Neighbourhoods', 'to_id': 'Category', 'travel_time': 'Value'}, inplace=True)

# Select and reorder columns
final_df = combined_df[['CTUID', 'Neighbourhoods', 'Metric_Type', 'Metric_Name', 'Travel_Time_Threshold', 'Category', 'Before_After_Benefit', 'Value']]

# Save to a new CSV file
final_df.to_csv('../../../data/visual_data/pivoted_commute_time_table.csv', index=False)

final_df

,CTUID,Neighbourhoods,Metric_Type,Metric_Name,Travel_Time_Threshold,Category,Before_After_Benefit,Value
0,5350128.04,South Eglinton-Davisville,Economic,Travel Time to Key Work Locations,32.0,Downtown Toronto,before,32.0
1,5350128.04,South Eglinton-Davisville,Economic,Travel Time to Key Work Locations,58.0,Etobicoke Centre,before,58.0
2,5350128.04,South Eglinton-Davisville,Economic,Travel Time to Key Work Locations,17.0,Yonge-Eglinton Centre,before,17.0
3,5350128.04,South Eglinton-Davisville,Economic,Travel Time to Key Work Locations,31.0,North York Centre,before,31.0
4,5350128.04,South Eglinton-Davisville,Economic,Travel Time to Key Work Locations,77.0,Scarborough Centre,before,77.0
...,...,...,...,...,...,...,...,...
2910,5350017.02,St Lawrence-East Bayfront-The Islands,Economic,Travel Time to Key Work Locations,4.0,Downtown Toronto,benefit,4.0
2911,5350017.02,St Lawrence-East Bayfront-The Islands,Economic,Travel Time to Key Work Locations,3.0,Etobicoke Centre,benefit,3.0
2912,5350017.02,St Lawrence-East Bayfront-The Islands,Economic,Travel Time to Key Work Locations,0.0,Yonge-Eglinton Centre,benefit,0.0
2913,5350017.02,St Lawrence-East Bayfront-The Islands,Economic,Travel Time to Key Work Locations,0.0,North York Centre,benefit,0.0


### CO2 emission calculation 
assumptions: 
1. 35% of the population drive to work. 
2. 10% of the people who drive to work will go to each of the five key work locations. 
3. If the public transit travel time reduction is bigger than 10 mins and if the new public transit travel time is within 60 mins, we believe 50% of the people who originally drive to work will switch to public transit.

Calculation steps:
1. Calculate public transit travel time to key work locations before and after the new line.

2. Calculate drive time to the key work locations.

3. Calculate CO2 emission (Driving to specific work locations)
Before the new transit line.
0.35 (percentage of people who drive to work) * 0.1 (percentage of people who will go to one of the key work locations) * 0.14167 kg (carbon emission per minute at 50 km/h)

After the new transit line.
Before the new line.
For those census tract that the public transit travel time reduction is bigger than 10 mins and the new public transit travel time is within 60 mins.

0.5*(0.35 (percentage of people who drive to work) * 0.1 (percentage of people who will go to one of the key work locations) * 0.14167 kg (carbon emission per minute at 50 km/h))

In [57]:
# Load the 'before drive' data
before_drive_df = pd.read_csv('../results/TTM_CT_Job_Before_Drive.csv')

filtered_drive_time_before = before_drive_df[before_drive_df['to_id'].isin(before_df['to_id'])]
filtered_drive_time_before.rename(columns={'travel_time': 'drive_time_before'}, inplace=True)
filtered_drive_time_before
result_with_drive_time_before = pd.merge(before_df, filtered_drive_time_before, on=['from_id', 'to_id'], how='left')
before_CO2 = result_with_drive_time_before[["from_id", "to_id", "travel_time", "drive_time_before"]]
before_CO2['CO2_Emission'] = before_CO2["drive_time_before"] * 0.35 * 0.1 * 0.14167
clean_CO2_before = before_CO2[['from_id', 'to_id', 'CO2_Emission']]
clean_CO2_before.rename(columns={'CO2_Emission': 'value'}, inplace=True)

/var/folders/7x/ttvgvv1x4yz0gh5wzclp2jp80000gn/T/ipykernel_1833/651792457.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_CO2['CO2_Emission'] = before_CO2["drive_time_before"] * 0.35 * 0.1 * 0.14167
/var/folders/7x/ttvgvv1x4yz0gh5wzclp2jp80000gn/T/ipykernel_1833/651792457.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_CO2_before.rename(columns={'CO2_Emission': 'value'}, inplace=True)


In [56]:
after_travel_df = after_df.copy()
after_travel_df.rename(columns={'travel_time': 'travel_time_after'}, inplace=True)
result_with_drive_time_after = pd.merge(after_travel_df, before_CO2, on=['from_id', 'to_id'], how='left')
after_CO2 = result_with_drive_time_after[["from_id", "to_id", "travel_time", "drive_time_before", "travel_time_after"]]
after_CO2.rename(columns={'travel_time': 'travel_time_before'}, inplace=True)
# 35% people drive, 10 % of them will go to this destination, 
after_CO2['CO2_Emission'] = after_CO2["drive_time_before"] * 0.35 * 0.1 * 0.14167
after_CO2["Travel_Time_Benefit"] = after_CO2["travel_time_before"] - after_CO2["travel_time_after"]

after_CO2.loc[(after_CO2["Travel_Time_Benefit"] > 10) & (after_CO2["travel_time_after"] < 60), 'CO2_Emission'] *= 0.5
clean_CO2_After = after_CO2[["from_id", "to_id","CO2_Emission"]]
clean_CO2_After.rename(columns={'CO2_Emission': 'value'}, inplace=True)
clean_CO2_After

/var/folders/7x/ttvgvv1x4yz0gh5wzclp2jp80000gn/T/ipykernel_1833/795890043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_CO2.rename(columns={'travel_time': 'travel_time_before'}, inplace=True)
/var/folders/7x/ttvgvv1x4yz0gh5wzclp2jp80000gn/T/ipykernel_1833/795890043.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_CO2['CO2_Emission'] = after_CO2["drive_time_before"] * 0.35 * 0.1 * 0.14167
/var/folders/7x/ttvgvv1x4yz0gh5wzclp2jp80000gn/T/ipykernel_1833/795890043.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,from_id,to_id,value
0,5350128.04,Downtown Toronto,0.054543
1,5350128.04,Etobicoke Centre,0.138837
2,5350128.04,Yonge-Eglinton Centre,0.019834
3,5350128.04,North York Centre,0.079335
4,5350128.04,Scarborough Centre,0.094211
...,...,...,...
2910,5350017.02,Downtown Toronto,0.024792
2911,5350017.02,Etobicoke Centre,0.094211
2912,5350017.02,Yonge-Eglinton Centre,0.074377
2913,5350017.02,North York Centre,0.114044


In [ ]:
import pandas as pd

# Load the CSV files
# before_df = pd.read_csv('../results/TTM_CT_Job_Before.csv')
# after_df = pd.read_csv('../results/TTM_CT_Job_After.csv')
neighbourhood_df = pd.read_csv('../../../data/visual_data/CTUID-w-Neighborhood.csv')

# Merge with neighbourhood data
before_CO2_df = clean_CO2_before.merge(neighbourhood_df, left_on='from_id', right_on='CTUID')
after_CO2_df = clean_CO2_After.merge(neighbourhood_df, left_on='from_id', right_on='CTUID')

# Calculate the benefit
before_CO2_df['Before_After_Benefit'] = 'before'
after_CO2_df['Before_After_Benefit'] = 'after'
combined_CO2_df = pd.concat([before_CO2_df, after_CO2_df])

# Calculate the benefit (before - after)
benefit_CO2_df = before_CO2_df.copy()
benefit_CO2_df['Before_After_Benefit'] = 'benefit'
benefit_CO2_df['value'] = before_CO2_df['value'] - after_CO2_df['value']
combined_CO2_df = pd.concat([combined_CO2_df, benefit_CO2_df])

# Add constant columns
combined_CO2_df['Metric_Type'] = 'Environmental'
combined_CO2_df['Metric_Name'] = 'CO2 Emission (Driving)'
combined_CO2_df['Travel_Time_Threshold'] = 0

# Rename columns
combined_CO2_df.rename(columns={'AREA_NAME': 'Neighbourhoods', 'to_id': 'Category', 'value': 'Value'}, inplace=True)

# Select and reorder columns
final_CO2_df = combined_CO2_df[['CTUID', 'Neighbourhoods', 'Metric_Type', 'Metric_Name', 'Travel_Time_Threshold', 'Category', 'Before_After_Benefit', 'Value']]

# Save to a new CSV file
final_CO2_df.to_csv('../../../data/visual_data/pivoted_CO2_table.csv', index=False)

final_CO2_df

,CTUID,Neighbourhoods,Metric_Type,Metric_Name,Travel_Time_Threshold,Category,Before_After_Benefit,Value
0,5350128.04,South Eglinton-Davisville,Environmental,CO2 Emission (Driving),0,Downtown Toronto,before,0.054543
1,5350128.04,South Eglinton-Davisville,Environmental,CO2 Emission (Driving),0,Etobicoke Centre,before,0.138837
2,5350128.04,South Eglinton-Davisville,Environmental,CO2 Emission (Driving),0,Yonge-Eglinton Centre,before,0.019834
3,5350128.04,South Eglinton-Davisville,Environmental,CO2 Emission (Driving),0,North York Centre,before,0.079335
4,5350128.04,South Eglinton-Davisville,Environmental,CO2 Emission (Driving),0,Scarborough Centre,before,0.094211
...,...,...,...,...,...,...,...,...
2910,5350017.02,St Lawrence-East Bayfront-The Islands,Environmental,CO2 Emission (Driving),0,Downtown Toronto,benefit,0.000000
2911,5350017.02,St Lawrence-East Bayfront-The Islands,Environmental,CO2 Emission (Driving),0,Etobicoke Centre,benefit,0.000000
2912,5350017.02,St Lawrence-East Bayfront-The Islands,Environmental,CO2 Emission (Driving),0,Yonge-Eglinton Centre,benefit,0.000000
2913,5350017.02,St Lawrence-East Bayfront-The Islands,Environmental,CO2 Emission (Driving),0,North York Centre,benefit,0.000000
